In [23]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import numpy as np
from numpy import where
from collections import Counter
from sklearn.datasets import make_blobs
import matplotlib

from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MultipleLocator, FormatStrFormatter 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib.patches import ConnectionPatch
from scipy.interpolate import make_interp_spline
from scipy.ndimage import gaussian_filter1d
%matplotlib inline

In [24]:
def get_acc(outputs, labels):
    """计算acc"""
    _, predict = torch.max(outputs.data, 1)
    total_num = labels.shape[0]*1.0
    correct_num = (labels == predict).sum().item()
    acc = correct_num / total_num

    return acc

# FGSM attack
def fgsm(model, X, y, epsilon):
    """ Construct FGSM adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    loss = nn.CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    #print("=== delta.grad ===\n", delta.grad)
    #print("=== delta.grad.sign ===\n",delta.grad.detach().sign())
    return epsilon * delta.grad.detach().sign()


In [25]:
#dataset and label

# size = 10  ###这里可以更改数据数量###
x= torch.unsqueeze(torch.from_numpy(np.append(np.random.normal(-1,2,5),np.random.normal(1,2,5))), dim=1)
y= torch.cat((torch.zeros(5),torch.ones(5))).type(torch.LongTensor)

dataset = Data.TensorDataset(x, y)
dataloader = Data.DataLoader(dataset=dataset, batch_size=10, shuffle=True)

In [26]:
#model
model = torch.nn.Sequential(
    nn.Linear(1, 10),
    nn.ReLU(),
    nn.Linear(10,2)
)


# optimizer
optim = torch.optim.Adam(model.parameters(), lr=0.1)

# Loss
loss_fun = nn.CrossEntropyLoss()

In [31]:
# training
for e in range(1000):
    epoch_loss = 0
    epoch_acc = 0
    
    



    # Perturbation
    delta = fgsm(model, x.float(), y, 0.1)   ###0.1 为扰动强度，可以更改
    xp = x + delta
    yp = model(x.float() + delta)

    num_epochs = 100

    # train
for e in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0
    for i, (x, y) in enumerate(dataloader):
        optim.zero_grad()

        out = model(xp.float())
        loss = loss_fun(out, y)

        loss.backward()




        optim.step()

        epoch_loss += loss.data
        epoch_acc += get_acc(out, y)

    if  e % 20 == 0:
        print('epoch: %d, loss: %f, acc: %f' % (e, epoch_loss , epoch_acc ))

epoch: 0, loss: 0.693422, acc: 0.500000
epoch: 20, loss: 0.693075, acc: 0.600000
epoch: 40, loss: 0.690331, acc: 0.600000
epoch: 60, loss: 0.686796, acc: 0.600000
epoch: 80, loss: 0.653121, acc: 0.600000
